plan:<br>
1) train model <br>
2) box importances - throw away useless features<br>
3) permutation importances<br>
4) feature analysis<br>
5) error analysis<br>
6) conclusions<br>

# Imports

In [5]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from utils import metrics
from utils import preprocessing as pr

# Load data

In [6]:
df = pd.read_csv('data/ready/train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,12,2008,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,0,62.0,7917,0,0,0,0,0,...,0,0,0,0,0,8,2007,0,0,175000
1456,1457,20,0,85.0,13175,0,0,0,0,0,...,0,0,1,0,0,2,2010,0,0,210000
1457,1458,70,0,66.0,9042,0,0,0,0,0,...,0,0,3,1,2500,5,2010,0,0,266500
1458,1459,20,0,68.0,9717,0,0,0,0,0,...,0,0,0,0,0,4,2010,0,0,142125


# Train-val split

In [7]:
df_train, df_val = train_test_split(df, test_size=0.2)
df_train.shape, df_val.shape

((1168, 81), (292, 81))

In [8]:
cols_x = df_train.drop(columns=['Id', 'SalePrice']).columns
cols_y = ['SalePrice']

# Init model

In [9]:
df_train['SalePrice'] = df_train['SalePrice'].apply(pr.one_plus_log)
df_val['SalePrice'] = df_val['SalePrice'].apply(pr.one_plus_log)

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_4078/475865074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['SalePrice'] = df_train['SalePrice'].apply(pr.one_plus_log)
/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_4078/475865074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['SalePrice'] = df_val['SalePrice'].apply(pr.one_plus_log)


In [12]:
rf = RandomForestRegressor(n_jobs=-1, random_state=27, max_features=0.25, n_estimators=300)
rf.fit(df_train[cols_x], df_train[cols_y])

y_train_hat = rf.predict(df_train[cols_x])
y_val_hat = rf.predict(df_val[cols_x])

print('Train RMSE', metrics.rmse(y_train_hat, df_train[cols_y]))
print('Val RMSE', metrics.rmse(y_val_hat, df_val[cols_y]))

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_4078/1413278653.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(df_train[cols_x], df_train[cols_y])


Train RMSE 0.05271432185878563
Val RMSE 0.12312560369245229


# Importances

## Box importances

1)

In [24]:
imp = dict(zip(cols_x, rf.feature_importances_))
min(imp.values()), max(imp.values()), len(imp.keys())

(2.2828070928518575e-06, 0.18167831137662052, 79)

In [25]:
to_keep = [k for k in imp.keys() if imp[k] > 10**-3]
len(to_keep)

61

In [26]:
rf = RandomForestRegressor(n_jobs=-1, random_state=27, max_features=0.25, n_estimators=300)
rf.fit(df_train[to_keep], df_train[cols_y])

y_train_hat = rf.predict(df_train[to_keep])
y_val_hat = rf.predict(df_val[to_keep])

print('Train RMSE', metrics.rmse(y_train_hat, df_train[cols_y]))
print('Val RMSE', metrics.rmse(y_val_hat, df_val[cols_y]))

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_4078/1513299012.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(df_train[to_keep], df_train[cols_y])


Train RMSE 0.05270945642467727
Val RMSE 0.12488553830423038


In [38]:
df_test = pd.read_csv('data/ready/test.csv')
df_test['SalePrice'] = rf.predict(df_test[to_keep])
df_test['SalePrice'] = df_test['SalePrice'].apply(pr.one_plus_log_reverse)
df_test[['Id', 'SalePrice']].to_csv('data/results/rf3.csv', index=False)

2)

In [27]:
imp = dict(zip(cols_x, rf.feature_importances_))
min(imp.values()), max(imp.values()), len(imp.keys())

(0.0008948898976000532, 0.18897546991261466, 61)

In [35]:
to_keep = [k for k in imp.keys() if imp[k] > 3 * 10**-3]
len(to_keep)

40

In [36]:
rf = RandomForestRegressor(n_jobs=-1, random_state=27, max_features=0.25, n_estimators=300)
rf.fit(df_train[to_keep], df_train[cols_y])

y_train_hat = rf.predict(df_train[to_keep])
y_val_hat = rf.predict(df_val[to_keep])

print('Train RMSE', metrics.rmse(y_train_hat, df_train[cols_y]))
print('Val RMSE', metrics.rmse(y_val_hat, df_val[cols_y]))

/var/folders/yl/z922qgpd7wq35049r7_cn54m0000gn/T/ipykernel_4078/1513299012.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(df_train[to_keep], df_train[cols_y])


Train RMSE 0.05430123381768714
Val RMSE 0.132530900104504
